# 모듈 + 불러오기

In [1]:
import pandas as pd
import numpy as np

In [2]:
from geoband.API import *
GetCompasData('SBJ_2208_001', '4', '4_연수구_독거노인.csv')

[2022-09-22 12:36:01] 성공 - 4_연수구_독거노인.csv


'4_연수구_독거노인.csv'

In [3]:
# 독거노인 : senior_alone
# 2017 ~ 2022 데이터

senior_alone = pd.read_csv('4_연수구_독거노인.csv')

In [4]:
senior_alone.head()

,emd_nm,tot_cnt,m_cnt,w_cnt,base_date
0,옥련1동,565,154,411,201712
1,옥련2동,415,115,300,201712
2,선학동,645,206,439,201712
3,연수1동,493,188,305,201712
4,연수2동,753,184,569,201712


# 전처리

In [5]:
# 날짜데이터 문자열로 변환
senior_alone['base_date'] = senior_alone['base_date'].astype('str')

In [6]:
# 연도만 추출
date_list = list(senior_alone['base_date'])
year = ['2017', '2018', '2019', '2020', '2021', '2022']

for i in range(len(date_list)):
    for j in range(len(year)):
        if date_list[i][:-2] == year[j]:
            date_list[i] = year[j]
            
senior_alone['base_date'] = date_list

In [7]:
senior_alone['base_date'].unique()

array(['2017', '2018', '2019', '2020', '2021', '2022'], dtype=object)

In [8]:
senior_alone['emd_nm'].unique()

array(['옥련1동', '옥련2동', '선학동 ', '연수1동', '연수2동', '연수3동', '청학동 ', '동춘1동',
       '동춘2동', '동춘3동', '송도1동', '송도2동', '송도3동', '송도4동', '송도5동', '선학동',
       '청학동'], dtype=object)

In [9]:
# 읍면동 명칭 수정
senior_alone['emd_nm'] = senior_alone['emd_nm'].apply(lambda x :'옥련동' if x in ['옥련1동', '옥련2동'] else x )
senior_alone['emd_nm'] = senior_alone['emd_nm'].apply(lambda x :'연수동' if x in ['연수1동', '연수2동', '연수3동'] else x )
senior_alone['emd_nm'] = senior_alone['emd_nm'].apply(lambda x :'동춘동' if x in ['동춘1동', '동춘2동', '동춘3동'] else x )
senior_alone['emd_nm'] = senior_alone['emd_nm'].apply(lambda x :'송도동' if x in ['송도1동', '송도2동', 
                                                                           '송도3동', '송도4동', '송도5동'] else x )
senior_alone['emd_nm'] = senior_alone['emd_nm'].apply(lambda x :'선학동' if x in ['선학동', '선학동 '] else x )
senior_alone['emd_nm'] = senior_alone['emd_nm'].apply(lambda x :'청학동' if x in ['청학동', '청학동 '] else x )

In [10]:
senior_alone['emd_nm'].unique()

array(['옥련동', '선학동', '연수동', '청학동', '동춘동', '송도동'], dtype=object)

In [11]:
# 연도 / 동 기준으로 groupby

# tot_cnt : 독거노인 총 합
# m_cnt : 남성 독거노인 수
# w_cnt : 여성 독거노인 수

senior_alone2 = senior_alone.copy()
senior_alone2 = senior_alone2.groupby(['base_date', 'emd_nm']).sum()
senior_alone2.reset_index(inplace = True)

# 최종 : 독거노인_1821

In [14]:
senior_alone2['base_date'] = senior_alone2['base_date'].astype(int)

In [21]:
# 18 ~ 21 년만 뽑기

senior_alone_1822 = senior_alone2[senior_alone2['base_date'] >= 2018]
senior_alone_1821 = senior_alone_1822[senior_alone_1822['base_date'] < 2022]
senior_alone_1821.reset_index(inplace = True)
senior_alone_1821 = senior_alone_1821.drop(columns = 'index')
senior_alone_1821.head()

,base_date,emd_nm,tot_cnt,m_cnt,w_cnt
0,2018,동춘동,705,181,524
1,2018,선학동,701,225,476
2,2018,송도동,641,188,453
3,2018,연수동,2296,639,1657
4,2018,옥련동,1066,287,779


In [23]:
# 저장
senior_alone_1821.to_csv('최종_독거노인_1821.csv', encoding = 'utf-8', index = False)

In [25]:
# 불러오기
senior_alone_fin = pd.read_csv('최종_독거노인_1821.csv', encoding = 'utf-8')
senior_alone_fin.head()

,base_date,emd_nm,tot_cnt,m_cnt,w_cnt
0,2018,동춘동,705,181,524
1,2018,선학동,701,225,476
2,2018,송도동,641,188,453
3,2018,연수동,2296,639,1657
4,2018,옥련동,1066,287,779


# 최종 : 독거노인_신도심_원도심

In [26]:
senior_alone_fin.head()

,base_date,emd_nm,tot_cnt,m_cnt,w_cnt
0,2018,동춘동,705,181,524
1,2018,선학동,701,225,476
2,2018,송도동,641,188,453
3,2018,연수동,2296,639,1657
4,2018,옥련동,1066,287,779


In [42]:
# 원도심
senior_alone_old = senior_alone_fin[senior_alone_fin['emd_nm'] != '송도동']
senior_alone_old = senior_alone_old.groupby('base_date').sum()
senior_alone_old.reset_index(inplace = True)
senior_alone_old.insert(1, 'center', '원도심')
senior_alone_old

,base_date,center,tot_cnt,m_cnt,w_cnt
0,2018,원도심,5792,1676,4116
1,2019,원도심,6436,1917,4519
2,2020,원도심,7168,2204,4964
3,2021,원도심,7743,2413,5330


In [47]:
# 신도심
senior_alone_new = senior_alone_fin[senior_alone_fin['emd_nm'] == '송도동']
senior_alone_new.reset_index(inplace = True)
senior_alone_new = senior_alone_new.drop(columns = 'index')
senior_alone_new = senior_alone_new.rename(columns = {'emd_nm' : 'center'})
senior_alone_new['center'] = '신도심'
senior_alone_new

,base_date,center,tot_cnt,m_cnt,w_cnt
0,2018,신도심,641,188,453
1,2019,신도심,772,229,543
2,2020,신도심,1839,992,847
3,2021,신도심,1158,339,819


In [58]:
# 합치기

senior_alone_old_new = pd.concat([senior_alone_old, senior_alone_new])
senior_alone_old_new = senior_alone_old_new.sort_values(by = senior_alone_old_new.columns[0])
senior_alone_old_new.reset_index(inplace = True)
senior_alone_old_new = senior_alone_old_new.drop(columns = 'index')
senior_alone_old_new

,base_date,center,tot_cnt,m_cnt,w_cnt
0,2018,원도심,5792,1676,4116
1,2018,신도심,641,188,453
2,2019,원도심,6436,1917,4519
3,2019,신도심,772,229,543
4,2020,원도심,7168,2204,4964
5,2020,신도심,1839,992,847
6,2021,원도심,7743,2413,5330
7,2021,신도심,1158,339,819


In [59]:
# 저장
senior_alone_old_new.to_csv('최종_독거노인_신도심_원도심.csv', encoding = 'utf-8', index = False)

In [60]:
# 불러오기
senior_old_new_fin = pd.read_csv('최종_독거노인_신도심_원도심.csv', encoding = 'utf-8')
senior_old_new_fin.head()

,base_date,center,tot_cnt,m_cnt,w_cnt
0,2018,원도심,5792,1676,4116
1,2018,신도심,641,188,453
2,2019,원도심,6436,1917,4519
3,2019,신도심,772,229,543
4,2020,원도심,7168,2204,4964


# 데이터 신청

In [61]:
from geoband.API import *
RequestCompasData('최종_독거노인_신도심_원도심.csv')
RequestCompasData('최종_독거노인_1821.csv')

[2022-09-22 12:49:39] 성공 - 정상적으로 신청되었습니다. 마이페이지내 데이터요청 항목에서 진행사항을 확인해주세요.
[2022-09-22 12:49:39] 성공 - 정상적으로 신청되었습니다. 마이페이지내 데이터요청 항목에서 진행사항을 확인해주세요.
